Занятие 8. Gradient Boosting Machine ver 2
================

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
import os
os.chdir("C:/Users/USER/Documents/Python/_AD_Python_part_1_Lectures/AD_Python_1_12_GBM/")


Задача.
Нужно научиться распознавать доход человека (больше $50000 в год или меньше) по его ответам в ходе переписи населения.<br>
https://archive.ics.uci.edu/ml/datasets/Adult

Описание данных:<br>
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

In [3]:

# Имена переменных
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

# Читаем и обрабатываем обучающую выборку
df = pd.read_csv('adult.data', header=None, names=columns, na_values=' ?')
# Удаляем колонку education (потому что есть уже закодированная колонка education-num)
df = df.drop('education', axis=1)
# Кодируем отклик значениями 0 и 1
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})
# удаляем строки с NA значениями
df = df.dropna()

# Читаем и обрабатываем тестовую выборку, как выше
test = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)
test = test.drop('education', axis=1)
test['income'] = test['income'].map({' <=50K.': 0, ' >50K.': 1})
test = test.dropna()

In [4]:
# Все хорошо?
df.head()

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


Классы сбалансированы? Посчитаем распределение наблюдений по классам.

In [5]:
df['income'].value_counts(normalize=True)

0    0.751078
1    0.248922
Name: income, dtype: float64

Разбивать данные на обучающее и тестовое множество не надо. 
Уже сделано: таблицы df и test. 
Бинаризуем признаки, измеренные в номинальной шкале (категориальные признаки).
В Python эта операция часто называется one-hot encoding.

In [6]:
X_train = pd.get_dummies(df).drop('income', axis=1)
y_train = df['income']

X_test = pd.get_dummies(test).drop('income', axis=1)
y_test = test['income']

In [7]:
#   После бинаризации смотрим на получившиеся переменные
X_test.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_ Federal-gov', 'workclass_ Local-gov',
       'workclass_ Private', 'workclass_ Self-emp-inc',
       'workclass_ Self-emp-not-inc', 'workclass_ State-gov',
       'workclass_ Without-pay', 'marital-status_ Divorced',
       'marital-status_ Married-AF-spouse',
       'marital-status_ Married-civ-spouse',
       'marital-status_ Married-spouse-absent',
       'marital-status_ Never-married', 'marital-status_ Separated',
       'marital-status_ Widowed', 'occupation_ Adm-clerical',
       'occupation_ Armed-Forces', 'occupation_ Craft-repair',
       'occupation_ Exec-managerial', 'occupation_ Farming-fishing',
       'occupation_ Handlers-cleaners', 'occupation_ Machine-op-inspct',
       'occupation_ Other-service', 'occupation_ Priv-house-serv',
       'occupation_ Prof-specialty', 'occupation_ Protective-serv',
       'occupation_ Sales', 'occupation_ Tech-support',
       'o

После бинаризации категориальных признаков оказывается, что в обучающем множестве содержится на один столбец больше. 

In [8]:
print(len(X_train.columns))
print(len(X_test.columns))


88
87


Оказывается, в тестовой выборке нет ни одного представителя Голландии 
в колонке ```native-county```.

In [9]:
# Приводим множество названий столбцов к типу set, находим разность двух множеств.
print(set(X_train.columns) - set(X_test.columns))
print(set(X_test.columns) - set(X_train.columns))

{'native-country_ Holand-Netherlands'}
set()


Исправляем. Создадим полный список всех колонок (из обучающего и тестового множеств) и переиндексируем колонки в соответствии с этим списком. В результате в колонке ```native-country_ Holand-Netherlands``` в тестовом множестве появятся NaN-значения, которые мы заменим на нули.

In [10]:
columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns=columns).fillna(0)
X_test = X_test.reindex(columns=columns).fillna(0)

Теперь наборы колонок в обучающем и тестовом множествах идентичны. 
Порядок колонок тоже совпадает. 
На всякий случай убедимся в этом. 

In [11]:
# Команда all проверяет, все ли значения из входного списка равны True
all(X_train.columns == X_test.columns)

True

Перейдём к созданию модели.
Хотя все еще можно (нужно!) многое улучшить.
Например, объединить некоторые колонки, где мало "1".

### GradientBoostingClassifier

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
#  from sklearn.metrics import classification_report
from sklearn import metrics

In [13]:
model = GradientBoostingClassifier(random_state=42,
                                   # Доля наблюдений в случайной подвыборке для очередного дерева
                                   subsample=0.66,
                                   # Доля переменных в случайной подвыборке для очередного дерева
                                   max_features='sqrt', 
                                   # Число деревьев
                                   n_estimators=500,
                                   #  критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’ (как в AdaBoost)
                                   #  ‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # shrinkage На это число умножаем каждое дерево.
                                   # Рекомендуется выставлять небольшие значения из (0, 0.3].
                                   learning_rate=0.01, 
                                   #  загрязнение дерева измеряем “mse” или “friedman_mse”  (mse с улучшениями)
                                    #  “mae” удалено из нынешней версии,  
                                   criterion='friedman_mse', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.001, 
                                   # минимальное число наблюдений в узле потомке
                                   min_samples_leaf=5, 
                                   # минимальное число наблюдений в узле родителе
                                   min_samples_split=10,
                                   # число узлов в дереве (в RandomForest иначе !!!!!!!!!!!!!)
                                   max_depth=5,
                                   # как выдавать промежуточные результаты
                                   verbose=0
                                   # Есть и другие параметры, уменьшающие размер дерева,
                                   # такие же как у DecisionTreeClassifier
                                   )


In [14]:
model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=5,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0001,
                           min_impurity_split=None, min_samples_leaf=5,
                           min_samples_split=10, min_weight_fraction_leaf=0.0,
                           n_estimators=500, n_iter_no_change=None,
                           presort='auto', random_state=42, subsample=0.66,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [15]:
y_pred_test = model.predict(X_test)
print(metrics.classification_report(y_test, y_pred_test))


              precision    recall  f1-score   support

           0       0.87      0.95      0.91     11360
           1       0.79      0.58      0.67      3700

    accuracy                           0.86     15060
   macro avg       0.83      0.76      0.79     15060
weighted avg       0.85      0.86      0.85     15060



In [16]:
y_pred_train = model.predict(X_train)
print(metrics.classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     22654
           1       0.80      0.58      0.68      7508

    accuracy                           0.86     30162
   macro avg       0.84      0.77      0.79     30162
weighted avg       0.86      0.86      0.85     30162



In [17]:
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,10803,557
1,1558,2142


In [18]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,21587,1067
1,3135,4373


In [19]:
print("Accuracy on train set is ", metrics.accuracy_score(y_train, y_pred_train)*100)
print("Accuracy on test set is ", metrics.accuracy_score(y_test, y_pred_test)*100)

Accuracy on train set is  86.06856309263311
Accuracy on test set is  85.95617529880478


Посмотрим на важность признаков.

In [20]:
fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

,features,importance
42,capital-gain,0.198692
52,education-num,0.140607
18,marital-status_ Married-civ-spouse,0.132698
66,relationship_ Husband,0.091909
26,age,0.073616
68,capital-loss,0.054231
34,hours-per-week,0.044382
54,marital-status_ Never-married,0.038326
33,occupation_ Exec-managerial,0.037087
10,relationship_ Wife,0.023860


# Калибровка

In [21]:
from sklearn.calibration import CalibratedClassifierCV

In [22]:
model_sigmoid = CalibratedClassifierCV(model, cv=2, method='sigmoid')
# метод калибровки: ‘sigmoid’ или ‘isotonic’

In [23]:
# Собственно калибровка
model_sigmoid.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                                 init=None,
                                                                 learning_rate=0.01,
                                                                 loss='deviance',
                                                                 max_depth=5,
                                                                 max_features='sqrt',
                                                                 max_leaf_nodes=None,
                                                                 min_impurity_decrease=0.0001,
                                                                 min_impurity_split=None,
                                                                 min_samples_leaf=5,
                                                                 min_samples_split=10,
                                                           

In [24]:
# После калибровки. Такими стали вероятности принадлежать классам
model_sigmoid.predict_proba(X_test)[0:11, :]

array([[0.99548296, 0.00451704],
       [0.78692195, 0.21307805],
       [0.7173267 , 0.2826733 ],
       [0.03406638, 0.96593362],
       [0.99158801, 0.00841199],
       [0.32880496, 0.67119504],
       [0.99473724, 0.00526276],
       [0.90369688, 0.09630312],
       [0.14473711, 0.85526289],
       [0.33551564, 0.66448436],
       [0.99309374, 0.00690626]])

In [25]:
zzz_1 = model_sigmoid.predict_proba(X_test)[:, :]

In [26]:
zzz_1[0:11, 0]

array([0.99548296, 0.78692195, 0.7173267 , 0.03406638, 0.99158801,
       0.32880496, 0.99473724, 0.90369688, 0.14473711, 0.33551564,
       0.99309374])

In [27]:
zzz_2 = zzz_1[:, 1] > 0.6

In [28]:
zzz_2[0:11]

array([False, False, False,  True, False,  True, False, False,  True,
        True, False])

In [30]:
zzz_2.astype(int)[0:11]

array([0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0])

In [32]:
w = 0.6

In [33]:
np.mean((model.predict_proba(X_test)[:, 1] > w).astype(int) == y_test)

0.8529880478087649